In [1]:
import pandas as pd

In [2]:
df1 = pd.read_csv('../core/data/lahman/mlb_data/Fielding.csv')
df2 = pd.read_csv('../core/data/lahman/mlb_data/People.csv')

In [3]:
df1.columns

Index(['playerID', 'yearID', 'stint', 'teamID', 'lgID', 'POS', 'G', 'GS',
       'InnOuts', 'PO', 'A', 'E', 'DP', 'PB', 'WP', 'SB', 'CS', 'ZR'],
      dtype='object')

In [4]:
fielding_columns = ['playerID', 'POS']
df1 = df1[fielding_columns]

In [5]:
df2.columns

Index(['playerID', 'birthYear', 'birthMonth', 'birthDay', 'birthCountry',
       'birthState', 'birthCity', 'deathYear', 'deathMonth', 'deathDay',
       'deathCountry', 'deathState', 'deathCity', 'nameFirst', 'nameLast',
       'nameGiven', 'weight', 'height', 'bats', 'throws', 'debut', 'finalGame',
       'retroID', 'bbrefID'],
      dtype='object')

In [6]:
people_columns = ['playerID', 'birthYear', 'bats', 'throws', 'weight', 'height']
df2 = df2[people_columns]

In [7]:
# This will be exported to a separate module - retroid_dict.py
ids = pd.read_csv('../core/data/lahman/mlb_data/People.csv')
ids = ids[['playerID', 'retroID']]
id_dict = ids.set_index('playerID').to_dict()['retroID']

def get_retroid(id):
    return id_dict[id] if id_dict is not None else ''

In [8]:
df1['playerID'] = df1['playerID'].apply(get_retroid)
df2['playerID'] = df2['playerID'].apply(get_retroid)
df1.rename(columns={'playerID': 'retroID'}, inplace=True)
df2.rename(columns={'playerID': 'retroID'}, inplace=True)

In [9]:
df1 = df1.groupby('retroID').agg(pd.Series.mode)

In [10]:
df1 = df1.reset_index()
df1

,retroID,POS
0,aardd001,P
1,aaroh101,OF
2,aarot101,1B
3,aased001,P
4,abada001,1B
...,...,...
15026,zupcb001,OF
15027,zupof101,C
15028,zuveg101,P
15029,zuvep001,SS


In [11]:
df2 = df2[df2['retroID'].notnull()]

In [12]:
df2

,retroID,birthYear,bats,throws,weight,height
0,omaro101,1891.0,R,R,155.0,69.0
1,steeb103,1894.0,B,L,175.0,70.0
2,fisht103,1886.0,R,R,170.0,69.0
3,kanef101,1895.0,L,R,175.0,71.0
4,kingl101,1894.0,R,R,150.0,70.0
...,...,...,...,...,...,...
17293,willj801,1847.0,NaN,NaN,NaN,NaN
17296,winkb801,1930.0,R,R,168.0,69.0
17297,wriga801,1842.0,NaN,NaN,NaN,NaN
17299,younn801,1840.0,NaN,NaN,NaN,NaN


In [13]:
df1.shape

(15031, 2)

In [17]:
df1[df1['retroID'].isin(df2['retroID'])].shape

(15031, 2)

In [18]:
df1.head()

,retroID,POS
0,aardd001,P
1,aaroh101,OF
2,aarot101,1B
3,aased001,P
4,abada001,1B


In [19]:
df2.head()

,retroID,birthYear,bats,throws,weight,height
0,omaro101,1891.0,R,R,155.0,69.0
1,steeb103,1894.0,B,L,175.0,70.0
2,fisht103,1886.0,R,R,170.0,69.0
3,kanef101,1895.0,L,R,175.0,71.0
4,kingl101,1894.0,R,R,150.0,70.0


In [20]:
df2[df2['retroID'] == 'aardd001']

,retroID,birthYear,bats,throws,weight,height
13110,aardd001,1981.0,R,R,215.0,75.0


In [21]:
df = pd.merge(df1, df2, on='retroID')

In [22]:
df

,retroID,POS,birthYear,bats,throws,weight,height
0,aardd001,P,1981.0,R,R,215.0,75.0
1,aaroh101,OF,1934.0,R,R,180.0,72.0
2,aarot101,1B,1939.0,R,R,190.0,75.0
3,aased001,P,1954.0,R,R,190.0,75.0
4,abada001,1B,1972.0,L,L,184.0,73.0
...,...,...,...,...,...,...,...
15026,zupcb001,OF,1966.0,R,R,220.0,76.0
15027,zupof101,C,1939.0,L,R,182.0,71.0
15028,zuveg101,P,1924.0,R,R,195.0,76.0
15029,zuvep001,SS,1958.0,R,R,173.0,72.0
